In [19]:
import yfinance as yf
import pandas as pd
import numpy as np

In [20]:
# Define the list of indices and equities
indices = ['^GSPC', '^FTSE', '^N225', '^GDAXI', '000001.SS']
equities = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META']

In [21]:
# Load the daily data for indices
index_data = yf.download(indices, start='2010-01-01')['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [22]:
# Create an empty DataFrame to store equity data
equity_data = pd.DataFrame()

# Retrieve data for each equity
for equity in equities:
    equity_ticker = yf.Ticker(equity)
    equity_history = equity_ticker.history(start='2010-01-01')
    equity_data[equity] = equity_history['Close']

# Convert equity_data to time zone-naive
equity_data = equity_data.tz_localize(None)

# Combine index and equity data
data = pd.concat([index_data, equity_data], axis=1)

# Calculate daily returns
returns = data.pct_change().dropna()

# Calculate cumulative returns
cumulative_returns = (1 + returns).cumprod()

# Calculate maximum drawdowns
roll_max = cumulative_returns.rolling(window=len(cumulative_returns), min_periods=1).max()
daily_drawdown = cumulative_returns / roll_max - 1.0
max_drawdown = daily_drawdown.min()

# Calculate volatility
volatility = returns.std() * np.sqrt(252)

# Calculate the Sharpe ratio
risk_free_rate = 0.0  # Assuming no risk-free rate for simplicity
daily_risk_free_rate = (1 + risk_free_rate) ** (1/252) - 1
excess_returns = returns - daily_risk_free_rate
sharpe_ratio = np.mean(excess_returns) / np.std(returns) * np.sqrt(252)

# Calculate the Sortino ratio
downside_returns = returns.copy()
downside_returns[downside_returns > 0] = 0
sortino_ratio = np.mean(excess_returns) / np.std(downside_returns) * np.sqrt(252)

c:\Python 3.10.2\lib\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [23]:
# Create tables for indices and equities
index_metrics = pd.DataFrame({'Cumulative Returns': cumulative_returns.iloc[-1],
                              'Volatility': volatility,
                              'Max Drawdown': max_drawdown,
                              'Sharpe Ratio': sharpe_ratio,
                              'Sortino Ratio': sortino_ratio},
                             index=indices)

equity_metrics = pd.DataFrame({'Cumulative Returns': cumulative_returns.iloc[-1, len(indices):],
                               'Volatility': volatility[len(indices):],
                               'Max Drawdown': max_drawdown[len(indices):],
                               'Sharpe Ratio': sharpe_ratio[len(indices):],
                               'Sortino Ratio': sortino_ratio[len(indices):]},
                              index=equities)

# Display the tables
print("Index Metrics:")
print(index_metrics)
print("\nEquity Metrics:")
print(equity_metrics)


Index Metrics:
           Cumulative Returns  Volatility  Max Drawdown  Sharpe Ratio  \
^GSPC                3.236500    0.170295     -0.339250      0.693000   
^FTSE                1.472562    0.153952     -0.366055      0.298716   
^N225                3.577661    0.200291     -0.317989      0.660840   
^GDAXI               2.595249    0.191743     -0.387794      0.534257   
000001.SS            1.400517    0.198933     -0.522997      0.249342   

           Sortino Ratio  
^GSPC           1.065486  
^FTSE           0.458608  
^N225           1.066130  
^GDAXI          0.841053  
000001.SS       0.371935  

Equity Metrics:
       Cumulative Returns  Volatility  Max Drawdown  Sharpe Ratio  \
AAPL            10.879682    0.283936     -0.437972      0.882816   
MSFT            13.449192    0.263834     -0.371485      0.999781   
AMZN            10.872107    0.319305     -0.561453      0.817552   
GOOGL            8.170369    0.266885     -0.443201      0.826215   
META             6.425

In [ ]:
index_metrics.to_csv('./Index.csv')
equity_metrics.to_csv('./Equity.csv')

## Indices
|                     | S&P 500 (GSPC) | FTSE 100 (FTSE) | Nikkei 225 (N225) | DAX (GDAXI) | SSE Composite Index (000001.SS) |
|---------------------|----------------|-----------------|-------------------|-------------|-------------------------------|
| Cumulative Returns  | 3.2365         | 1.472562        | 3.577661          | 2.595249    | 1.400517                      |
| Volatility          | 0.170295       | 0.153952        | 0.200291          | 0.191743    | 0.198933                      |
| Max Drawdown        | -0.33925       | -0.36606        | -0.31799          | -0.38779    | -0.523                        |
| Sharpe Ratio        | 0.693          | 0.298716        | 0.66084           | 0.534257    | 0.249342                      |
| Sortino Ratio       | 1.065486       | 0.458608        | 1.06613           | 0.841053    | 0.371935                      |


## Equity

|                   | AAPL         | MSFT         | AMZN         | GOOGL        | META         |
|-------------------|--------------|--------------|--------------|--------------|--------------|
| Cumulative Returns| 10.87968223  | 13.44919227  | 10.8721065   | 8.170369043  | 6.42532049   |
| Volatility        | 0.283936354  | 0.263834026  | 0.319304597  | 0.26688514   | 0.400144226  |
| Max Drawdown      | -0.437971701 | -0.371484853 | -0.561452633 | -0.443200514 | -0.767360915 |
| Sharpe Ratio      | 0.882815832  | 0.999781308  | 0.817551739  | 0.826214825  | 0.609892954  |
| Sortino Ratio     | 1.449057739  | 1.665967534  | 1.375846545  | 1.393160141  | 0.997870649  |